In [1]:
import pandas as pd

In [2]:
#pd.set_option('display.max_columns', None)

clean_file = pd.read_csv('output/listing_details_cleaned.csv')
clean_file

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,Đường phố,Chi tiết,Nguồn thông tin,Tình trạng giao dịch,Thời điểm giao dịch/rao bán,Thông tin liên hệ,Giá rao bán/giao dịch,...,Kích thước chiều dài (m),Số mặt tiền tiếp giáp,Hình dạng,Độ rộng ngõ/ngách nhỏ nhất (m),Khoảng cách tới trục đường chính (m),Mục đích sử dụng đất,Yếu tố khác,Tọa độ (vĩ độ),Tọa độ (kinh độ),Hình ảnh của bài đăng
0,Khánh Hòa,Nha Trang,Phường Phương Sơn,Đường Hải Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,2.800000e+09,...,9.0,1,Nở hậu,NaN,NaN,Đất ở,"vị trí nhà tiếp giáp hẻm rộng 3m, cách đường 2...",12.252321,109.178597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
1,Hà Nội,Nam Từ Liêm,Phường Tây Mỗ,Đường Tây Mỗ,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,5.700000e+09,...,NaN,1,Chữ nhật,NaN,NaN,Đất ở,"nhà dân xây, phân lô, cần bán gấp giá rẻ | nhà...",21.004792,105.749786,"[""https://file4.batdongsan.com.vn/crop/600x315..."
2,Hồ Chí Minh,Phú Nhuận,Phường 5,Đường Thích Quảng Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,1.050000e+10,...,19.0,1,Chữ nhật,5.0,NaN,Đất ở,hẻm 5m ô tô vào nhà quay đầu - khu dân trí cao...,10.805619,106.682205,"[""https://file4.batdongsan.com.vn/crop/600x315..."
3,Hà Nội,Cầu Giấy,Phường Quan Hoa,Đường Nguyễn Khánh Toàn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,02/07/2025,NaN,1.820000e+10,...,NaN,1,Chữ nhật,3.5,NaN,Đất ở,"2 tỷ, phân lô, đường nguyễn khánh toàn, cầu gi...",21.037317,105.800733,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4,Hồ Chí Minh,Bình Tân,Phường Tân Tạo,Đường Lê Đình Cẩn,302/,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,02/07/2025,NaN,NaN,...,12.0,1,Nở hậu,6.0,NaN,Đất ở,13 dài 12 - lê đình cẩn - phường tân tạo - quậ...,10.759390,106.599232,"[""https://file4.batdongsan.com.vn/crop/600x315..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,Hà Nội,Hà Đông,Phường Quang Trung,Đường Ngô Thì Nhậm,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,1.000000e+10,...,50.0,1,Chữ nhật,NaN,NaN,Đất ở,+ nhà vị trí cực đẹp ngô thì nhậm sát đường qu...,20.968794,105.769142,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4986,Hà Nội,Đống Đa,Phường Ô Chợ Dừa,Phố Xã Đàn,Mặt phố,https://batdongsan.com.vn/ban-nha-rieng-pho-xa...,Rao bán,06/07/2025,NaN,8.700000e+09,...,44.0,1,Chữ nhật,NaN,20.0,Đất ở,thang máy xịn sò | víp đống đa - 6 tầng thang ...,21.015794,105.832597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4987,Hồ Chí Minh,Thủ Đức,Phường Trường Thọ,Đường Đặng Văn Bi,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,5.680000e+09,...,NaN,1,Nở hậu,5.0,NaN,Đất ở,"nhà 2 tầng mới keng, xây kiên cố, gồm 3 phòng ...",10.843900,106.762166,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4988,Hồ Chí Minh,Bình Thạnh,Phường 1,Đường Vũ Huy Tấn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,NaN,...,17.0,1,Chữ nhật,NaN,NaN,Đất ở,"đang cho thuê 22triệu/tháng, gần chung cư vũ h...",10.796153,106.693391,"[""https://file4.batdongsan.com.vn/crop/600x315..."


In [3]:
clean_file[clean_file['Độ rộng ngõ/ngách nhỏ nhất (m)'].isna()]

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,Đường phố,Chi tiết,Nguồn thông tin,Tình trạng giao dịch,Thời điểm giao dịch/rao bán,Thông tin liên hệ,Giá rao bán/giao dịch,...,Kích thước chiều dài (m),Số mặt tiền tiếp giáp,Hình dạng,Độ rộng ngõ/ngách nhỏ nhất (m),Khoảng cách tới trục đường chính (m),Mục đích sử dụng đất,Yếu tố khác,Tọa độ (vĩ độ),Tọa độ (kinh độ),Hình ảnh của bài đăng
0,Khánh Hòa,Nha Trang,Phường Phương Sơn,Đường Hải Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,2.800000e+09,...,9.0,1,Nở hậu,NaN,NaN,Đất ở,"vị trí nhà tiếp giáp hẻm rộng 3m, cách đường 2...",12.252321,109.178597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
1,Hà Nội,Nam Từ Liêm,Phường Tây Mỗ,Đường Tây Mỗ,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,5.700000e+09,...,NaN,1,Chữ nhật,NaN,NaN,Đất ở,"nhà dân xây, phân lô, cần bán gấp giá rẻ | nhà...",21.004792,105.749786,"[""https://file4.batdongsan.com.vn/crop/600x315..."
6,Hồ Chí Minh,Nhà Bè,Xã Phước Lộc,Đường Đào Sư Tích,592,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,7.900000e+09,...,NaN,2,Chữ nhật,NaN,NaN,Đất ở,NaN,10.695113,106.682383,"[""https://file4.batdongsan.com.vn/crop/600x315..."
7,Hà Nội,Hoài Đức,Xã An Khánh,Đường Ngãi Cầu,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,29/06/2025,NaN,4.300000e+09,...,NaN,2,Chữ nhật,NaN,NaN,Đất ở,"- nhà 2 mặt thoáng, nhà còn rất mới về ở luôn,...",20.989422,105.719030,"[""https://file4.batdongsan.com.vn/crop/600x315..."
8,Hồ Chí Minh,Quận 1,Phường Bến Thành,Đường Lê Thánh Tôn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,2.870000e+10,...,16.5,1,Nở hậu,NaN,NaN,Đất ở,"căn nhà 5 tầng đối new world 5 tầng, 9pn, 10wc...",10.773502,106.697876,"[""https://file4.batdongsan.com.vn/crop/600x315..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,Hà Nội,Bắc Từ Liêm,Phường Xuân Đỉnh,Đường Xuân Đỉnh,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,03/07/2025,NaN,9.600000e+09,...,NaN,2,Chữ nhật,NaN,NaN,Đất ở,"mặt tiền đẹp, rất thích hợp cho ai muốn mở cửa...",21.071558,105.794709,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4985,Hà Nội,Hà Đông,Phường Quang Trung,Đường Ngô Thì Nhậm,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,1.000000e+10,...,50.0,1,Chữ nhật,NaN,NaN,Đất ở,+ nhà vị trí cực đẹp ngô thì nhậm sát đường qu...,20.968794,105.769142,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4986,Hà Nội,Đống Đa,Phường Ô Chợ Dừa,Phố Xã Đàn,Mặt phố,https://batdongsan.com.vn/ban-nha-rieng-pho-xa...,Rao bán,06/07/2025,NaN,8.700000e+09,...,44.0,1,Chữ nhật,NaN,20.0,Đất ở,thang máy xịn sò | víp đống đa - 6 tầng thang ...,21.015794,105.832597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4988,Hồ Chí Minh,Bình Thạnh,Phường 1,Đường Vũ Huy Tấn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,NaN,...,17.0,1,Chữ nhật,NaN,NaN,Đất ở,"đang cho thuê 22triệu/tháng, gần chung cư vũ h...",10.796153,106.693391,"[""https://file4.batdongsan.com.vn/crop/600x315..."


In [4]:
clean_file.iloc[4989]['Nguồn thông tin']

'https://batdongsan.com.vn/ban-nha-rieng-duong-truong-dinh-phuong-tuong-mai/ban-pho-30m2-x-5-tang-4-86-ty-pr43126422'

In [5]:
clean_file.iloc[91]['Đường phố']

'Đường Tân Sơn Nhì'

In [6]:
dp = clean_file.iloc[91]['Đường phố']
if dp in clean_file.iloc[91]['Chi tiết']:
    print(clean_file.iloc[91]['Chi tiết'].replace(dp, ''))

Hẻm 84 


In [7]:
chua_clean = pd.read_csv('output\listing_details.csv')

<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Admin\AppData\Local\Temp\ipykernel_376\3281314335.py:1: SyntaxWarning: invalid escape sequence '\l'
  chua_clean = pd.read_csv('output\listing_details.csv')


In [14]:
for i in range(0, 11):
    heyo = chua_clean.iloc[i]['description']
    new_heyo = heyo.split('\n')
    print(heyo)
    print('\n End Des\n')
    new_version = []
    for part in new_heyo:
        if 'liên hệ' in part.lower() or 'lh' in part.lower():
            continue
        elif '***' in part.lower():
            continue
        elif 'tỷ' in part.lower().split() or 'tr' in part.lower().split() or 'triệu' in part.lower().split() and 'thương lượng' not in part.lower() and 'tốt' not in part.lower().split():
            continue
        elif 'giá' in part.lower().split() and 'thương lượng' not in part.lower() and 'tốt' not in part.lower().split():
            continue
        elif 'xem nhà' in part.lower():
            continue
        else:
            new_version.append(part)
    new_version = '\n'.join(new_version)
    print(f'New version:\n{new_version}\n\n')

Nhà gồm 1 trệt + 1 lửng + 1 lầu, thiết kế 4 phòng ngủ và 3 toilet, sân để xe rộng.
Vị trí nhà tiếp giáp hẻm rộng 3m, cách đường 23/10 chỉ 70m. Khu vực dân cư đông đúc, dân trí cao.
Diện tích: 48m². Ngang 3,5 x dài 9m x nở hậu 8m.
Sổ hồng chính chủ. Hướng Tây Nam.
Giá bán: 2 tỷ 800 triệu.
Vui lòng liên hệ anh Tám:
0901 904 ***
.

 End Des

New version:
Nhà gồm 1 trệt + 1 lửng + 1 lầu, thiết kế 4 phòng ngủ và 3 toilet, sân để xe rộng.
Vị trí nhà tiếp giáp hẻm rộng 3m, cách đường 23/10 chỉ 70m. Khu vực dân cư đông đúc, dân trí cao.
Diện tích: 48m². Ngang 3,5 x dài 9m x nở hậu 8m.
Sổ hồng chính chủ. Hướng Tây Nam.
.


Nhà Diện tích 52m rộng rãi thoáng mát ở được gia đình đông người
- Ngõ trước nhà 2,2 m, 100m ra oto dừng đỗ, 200m ra trường- chợ.
- Vị trí trung tâm, gần ngay Vinhome smart city, Aoenl Mall Hà Đông
* Thiết kế
- Tầng 1: Khách + bếp + wc
- Tầng 2,3: mỗi tầng 2 ngủ + wc
- Tầng 4: phòng thờ + sân phơi
Sổ vuông vức không lỗi.

 End Des

New version:
Nhà Diện tích 52m rộng rãi thoá